In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 27 kB/s 


In [ ]:
from catboost import CatBoostClassifier
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
wrong_samples = pd.read_csv('drive/My Drive/wrong_samples_color.csv', index_col=0)
true_samples = pd.read_csv('drive/My Drive/true_samples_color.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
wrong_samples.columns = ['text', 'class']
true_samples.columns = ['text', 'class']

In [ ]:
p = 0.1
idx = np.random.choice([True, False], p=[p, 1-p], size=len(true_samples))
true_samples = true_samples[idx]

In [ ]:
wrong_samples['class'] = 1
true_samples['class'] = 0
wrong_samples = wrong_samples.append(true_samples)

In [ ]:
wrong_samples.columns = ['text', 'class']

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 8.7 MB/s 
     |████████████████████████████████| 371 kB 51.5 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=870904 sha256=aae11ee91dbcbdf59d52fbd9aa09a1b37f3e0f2d8f7ab0a0d1097e8ffcbab693
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [ ]:
from pymorphy2 import MorphAnalyzer
morph_analyzer = MorphAnalyzer()
stop_words = stopwords.words("russian")
import re

In [ ]:
def preprocess(text):
  text = text.lower()
  text = re.sub('\s+', ' ', re.sub(r'[^А-Яа-яёЁ]', ' ', text).strip())
  text = [morph_analyzer.parse(word)[0].normal_form for word in text.split()]
  text = [word for word in text if word not in stop_words]
  return ' '.join(text)

In [ ]:
wrong_samples = wrong_samples[wrong_samples['text'].apply(type) == str]

In [ ]:
wrong_samples['text'] = wrong_samples['text'].apply(preprocess)

In [ ]:
wrong_samples['text']

0          административный регламент предоставление госу...
1          итог рассмотрение вопрос указанный абзац второ...
2          установить сведение представить гражданский сл...
3          б установить сведение представить гражданский ...
4          итог рассмотрение вопрос указанный абзац трети...
                                 ...                        
2520019                             наименование собственник
2520023                       вид право владение пользование
2520024                                характеристика здание
2520025        жилой нежилой административный складской иной
2520032                                                     
Name: text, Length: 217634, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
X = vectorizer.fit_transform(wrong_samples['text'].values.astype('U'))
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
from sklearn.decomposition import TruncatedSVD
from catboost.utils import get_confusion_matrix

In [ ]:
svd = TruncatedSVD(100)
X_svd = svd.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_svd, X_test_svd, y_train_svd, y_test_svd = train_test_split(X_svd, wrong_samples['class'], stratify=wrong_samples['class'], test_size=0.25)

In [ ]:
grid = {
        'depth': [4, 6, 8],
        'l2_leaf_reg': [0.1, 0.5, 1],
        'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
        'auto_class_weights': [None, 'Balanced', 'SqrtBalanced']
}

In [ ]:
model = CatBoostClassifier(eval_metric='F1', iterations=100, verbose=25, task_type='GPU')
grid_search_result = model.grid_search(grid, 
                                       X=X_train_svd, 
                                       y=y_train_svd,
                                       plot=True,
                                       verbose=10)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6578770	test: 0.6648350	best: 0.6648350 (0)	total: 75.8ms	remaining: 7.5s
25:	learn: 0.6698185	test: 0.6744504	best: 0.6750475 (24)	total: 1.72s	remaining: 4.89s
50:	learn: 0.6834305	test: 0.6837200	best: 0.6837200 (50)	total: 3.29s	remaining: 3.16s
75:	learn: 0.6909800	test: 0.6913033	best: 0.6913033 (75)	total: 4.84s	remaining: 1.53s
99:	learn: 0.6955726	test: 0.6949282	best: 0.6949282 (99)	total: 6.3s	remaining: 0us
bestTest = 0.6949282096
bestIteration = 99
0:	loss: 0.6949282	best: 0.6949282 (0)	total: 8.82s	remaining: 11m 45s
0:	learn: 0.6578770	test: 0.6648350	best: 0.6648350 (0)	total: 87ms	remaining: 8.61s
25:	learn: 0.6728830	test: 0.6763658	best: 0.6763658 (25)	total: 1.58s	remaining: 4.51s
50:	learn: 0.6852067	test: 0.6860357	best: 0.6860357 (50)	total: 3.12s	remaining: 3s
75:	learn: 0.6916978	test: 0.6926236	best: 0.6926236 (75)	total: 4.63s	remaining: 1.46s
99:	learn: 0.6959456	test: 0.6955914	best: 0.6955914 (99)	total: 6.09s	remaining: 0us
bestTest = 0.695591

In [ ]:
#bin_params = grid_search_result['params']
bin_params = {'auto_class_weights': 'SqrtBalanced',
              'depth': 8,
              'grow_policy': 'Depthwise'
              }

In [ ]:
bin_params

{'auto_class_weights': 'SqrtBalanced', 'depth': 8, 'grow_policy': 'Depthwise'}

In [ ]:
model = CatBoostClassifier(eval_metric='F1', iterations=300, **bin_params)
model.fit(X_train_svd, y_train_svd, eval_set=(X_test_svd, y_test_svd))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.18833
0:	learn: 0.1081363	test: 0.0918956	best: 0.0918956 (0)	total: 660ms	remaining: 3m 17s
1:	learn: 0.0835182	test: 0.0743767	best: 0.0918956 (0)	total: 1.29s	remaining: 3m 11s
2:	learn: 0.1211223	test: 0.0980251	best: 0.0980251 (2)	total: 1.91s	remaining: 3m 9s
3:	learn: 0.1524680	test: 0.1125768	best: 0.1125768 (3)	total: 2.54s	remaining: 3m 8s
4:	learn: 0.1739373	test: 0.1308747	best: 0.1308747 (4)	total: 3.22s	remaining: 3m 10s
5:	learn: 0.1828965	test: 0.1382386	best: 0.1382386 (5)	total: 3.88s	remaining: 3m 10s
6:	learn: 0.1951069	test: 0.1542944	best: 0.1542944 (6)	total: 4.48s	remaining: 3m 7s
7:	learn: 0.2032581	test: 0.1588902	best: 0.1588902 (7)	total: 5.1s	remaining: 3m 6s
8:	learn: 0.2202174	test: 0.1722851	best: 0.1722851 (8)	total: 5.76s	remaining: 3m 6s
9:	learn: 0.2365280	test: 0.1828149	best: 0.1828149 (9)	total: 6.36s	remaining: 3m 4s
10:	learn: 0.2553239	test: 0.1924886	best: 0.1924886 (10)	total: 7.02s	remaining: 3m 4s
11:	learn: 0.2678510

In [ ]:
model.save_model("/content/drive/MyDrive/bin_model.cbm")

In [ ]:
from catboost import Pool
get_confusion_matrix(model, Pool(X_test_svd, y_test_svd))

array([[47898.,  1737.],
       [ 2834.,  1940.]])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as prfs
y_pred = model.predict(X_test_svd)
prfs(y_test_svd, y_pred)

(array([0.94413782, 0.52760403]),
 array([0.96500453, 0.40636783]),
 array([0.95445714, 0.45911726]),
 array([49635,  4774]))

In [ ]:
wrong_samples = pd.read_csv('drive/My Drive/wrong_samples_color.csv', index_col=0)
wrong_samples.columns = ['text', 'class']
wrong_samples = wrong_samples[wrong_samples['text'].apply(type) == str]
wrong_samples['text'] = wrong_samples['text'].apply(preprocess)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
X = vectorizer.fit_transform(wrong_samples['text'].values.astype('U'))
svd = TruncatedSVD(100)
X_svd = svd.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_svd, wrong_samples['class'], stratify=wrong_samples['class'], test_size=0.25)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
model = CatBoostClassifier(eval_metric='Accuracy', iterations=100, verbose=25, task_type='GPU')
grid_search_result_multi = model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train,
                                       plot=True,
                                       verbose=10)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.1912972	test: 0.1324809	best: 0.1324809 (0)	total: 15.6ms	remaining: 1.54s
25:	learn: 0.4009151	test: 0.2157960	best: 0.2427378 (24)	total: 402ms	remaining: 1.14s
50:	learn: 0.4531266	test: 0.2307940	best: 0.2427378 (24)	total: 793ms	remaining: 762ms
75:	learn: 0.4952143	test: 0.2707874	best: 0.2707874 (75)	total: 1.18s	remaining: 371ms
99:	learn: 0.5178150	test: 0.2799490	best: 0.2832482 (86)	total: 1.53s	remaining: 0us
bestTest = 0.2832482201
bestIteration = 86
0:	loss: 0.2832482	best: 0.2832482 (0)	total: 2.88s	remaining: 3m 50s
0:	learn: 0.1912972	test: 0.1324809	best: 0.1324809 (0)	total: 12.6ms	remaining: 1.25s
25:	learn: 0.4001156	test: 0.2155190	best: 0.2415914 (24)	total: 332ms	remaining: 945ms
50:	learn: 0.4501296	test: 0.2283518	best: 0.2415914 (24)	total: 648ms	remaining: 622ms
75:	learn: 0.4899425	test: 0.2696726	best: 0.2696726 (75)	total: 939ms	remaining: 297ms
99:	learn: 0.5139043	test: 0.2782755	best: 0.2846847 (94)	total: 1.24s	remaining: 0us
bestTest = 0.

In [ ]:
model.save_model("/content/drive/MyDrive/multi_model.cbm")

In [ ]:
multi_params = grid_search_result_multi['params']

In [ ]:
multi_params

{'auto_class_weights': 'Balanced',
 'depth': 8,
 'grow_policy': 'Depthwise',
 'l2_leaf_reg': 0.1}

In [ ]:
model = CatBoostClassifier(eval_metric='Accuracy', iterations=300)
model.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.180379
0:	learn: 0.3130368	test: 0.3028907	best: 0.3028907 (0)	total: 407ms	remaining: 2m 1s
1:	learn: 0.3226032	test: 0.3081274	best: 0.3081274 (1)	total: 818ms	remaining: 2m 1s
2:	learn: 0.3315411	test: 0.3148303	best: 0.3148303 (2)	total: 1.24s	remaining: 2m 2s
3:	learn: 0.3455066	test: 0.3288647	best: 0.3288647 (3)	total: 1.67s	remaining: 2m 3s
4:	learn: 0.3573075	test: 0.3355677	best: 0.3355677 (4)	total: 2.11s	remaining: 2m 4s
5:	learn: 0.3652678	test: 0.3464600	best: 0.3464600 (5)	total: 2.59s	remaining: 2m 7s
6:	learn: 0.3739264	test: 0.3508588	best: 0.3508588 (6)	total: 3.01s	remaining: 2m 5s
7:	learn: 0.3776971	test: 0.3558860	best: 0.3558860 (7)	total: 3.42s	remaining: 2m 5s
8:	learn: 0.3833531	test: 0.3617512	best: 0.3617512 (8)	total: 3.84s	remaining: 2m 4s
9:	learn: 0.3849592	test: 0.3665689	best: 0.3665689 (9)	total: 4.26s	remaining: 2m 3s
10:	learn: 0.3874729	test: 0.3686636	best: 0.3686636 (10)	total: 4.66s	remaining: 2m 2s
11:	learn: 0.3928497	t

In [ ]:
model.save_model("/content/drive/MyDrive/multi_model.cbm")

In [ ]:
y_pred = model.predict(X_test)
metrics = prfs(y_test, y_pred)

In [ ]:
metrics

(array([0.36809816, 0.41025641, 0.4801444 , 0.75      , 0.48791541,
        1.        , 0.50184957, 0.        , 0.39240506, 0.5228833 ,
        0.56985294, 0.23529412]),
 array([0.28526149, 0.16494845, 0.4149766 , 0.2       , 0.60373832,
        0.2       , 0.69453925, 0.        , 0.21088435, 0.62688615,
        0.36299766, 0.05797101]),
 array([0.32142857, 0.23529412, 0.44518828, 0.31578947, 0.53968254,
        0.33333333, 0.58267717, 0.        , 0.27433628, 0.57018091,
        0.4434907 , 0.09302326]),
 array([ 631,   97,  641,   15,  535,   15, 1172,    2,  441,  729,  427,
          69]))

In [ ]:
for c in range(len(metrics[0])):
  print("Class {}, Precision: {:1.4f}, Recall: {:1.4f}, F1: {:1.4f}".
        format(c, metrics[0][c], metrics[1][c], metrics[2][c]))

Class 0, Precision: 0.3681, Recall: 0.2853, F1: 0.3214
Class 1, Precision: 0.4103, Recall: 0.1649, F1: 0.2353
Class 2, Precision: 0.4801, Recall: 0.4150, F1: 0.4452
Class 3, Precision: 0.7500, Recall: 0.2000, F1: 0.3158
Class 4, Precision: 0.4879, Recall: 0.6037, F1: 0.5397
Class 5, Precision: 1.0000, Recall: 0.2000, F1: 0.3333
Class 6, Precision: 0.5018, Recall: 0.6945, F1: 0.5827
Class 7, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 8, Precision: 0.3924, Recall: 0.2109, F1: 0.2743
Class 9, Precision: 0.5229, Recall: 0.6269, F1: 0.5702
Class 10, Precision: 0.5699, Recall: 0.3630, F1: 0.4435
Class 11, Precision: 0.2353, Recall: 0.0580, F1: 0.0930
